In [44]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview.generative_models import GenerativeModel, ChatSession, SafetySetting


model = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    safety_settings=[
        SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="BLOCK_LOW_AND_ABOVE"),
        SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="BLOCK_LOW_AND_ABOVE"),
        SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="BLOCK_LOW_AND_ABOVE"),
        SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="BLOCK_LOW_AND_ABOVE"),
    ],
)

def classifyQuestion(question):
    prompt=f"""
      Classify the following question into one of the following categories: Employment, General Information, Emergency Services, or Tax Related
      Output only one of the category.
      question: {question}
    """
    return model.generate_content(prompt).text.strip();

def generateSocialMediaPosts(question):
    prompt=f"""
      generates social media posts for government announcements like weather emergencies, holidays, school closings, etc
      It has to be like a official post made by government standards.
      The post should be below 100 characters.
      Avoid anything other than government official posts.
      question: {question}
    """
    return model.generate_content(prompt).text.strip();

In [20]:
import unittest

class TestClassifyQuestion(unittest.TestCase):
    def test_EmploymentFlow(self):
      response = classifyQuestion("Where can i get job related to software development")
      self.assertEqual(response, "Employment")

    def test_GeneralFlow(self):
      response = classifyQuestion("What's the date ?")
      self.assertEqual(response, "General Information")

    def test_EmergencyFlow(self):
      response = classifyQuestion("What is the contact of fire station")
      self.assertEqual(response, "Emergency Services")

    def test_TaxFlow(self):
      response = classifyQuestion("What is the income tax percentage in US")
      self.assertEqual(response, "Tax Related")

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 1.246s

OK


In [23]:
import unittest

def does_post_follow_rules(post):
  response = model.generate_content(
    f"""Does the post follow the following rules:
    1. Keep your post below 100 characters
    2. Include the hashtag #EatAtLuigis at the end of every tweet
    Only return Yes or No
    post: {post}
    """
    )
  return response.text.strip()

class TestGenerateSocialMediaPosts(unittest.TestCase):
    def test_PositiveFlow(self):
      response = generateSocialMediaPosts("holiday due to covid")
      correct = does_post_follow_rules(response)
      self.assertEqual(correct, "Yes")

    def test_PositiveFlow(self):
      response = "Where can i get job related to software development"
      correct = does_post_follow_rules(response)
      self.assertEqual(correct, "No")

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 1.188s

OK


In [56]:
import vertexai
from vertexai.evaluation import EvalTask
from vertexai.generative_models import GenerativeModel
import pandas as pd
from vertexai.preview.evaluation import notebook_utils


PROJECT_ID='qwiklabs-gcp-04-c85702fbf940'
LOCATION='us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [57]:
posts=[
    {
        "prompt": "The annual cultural festival 'Utsav' in Kolkata will feature classical dance performances, music concerts, and art exhibitions from October 5 to October 10 at the Rabindra Sadan.",
        "post": "Kolkata's 'Utsav' cultural festival is back! Enjoy classical dance, music, and art at Rabindra Sadan from Oct 5-10. Don't miss out! #KolkataUtsav #CulturalFestival"
    },
    {
        "prompt": "A major power outage is expected in parts of Mumbai, including Bandra and Juhu, due to urgent cable repairs from 9 AM to 2 PM on July 1.",
        "post": "Heads up Mumbai! Power outage in Bandra & Juhu on July 1 from 9 AM - 2 PM for urgent repairs. Plan accordingly! #MumbaiPowerCut #ElectricityUpdate"
    },
    {
        "prompt": "The State Bank of India has revised its fixed deposit interest rates, effective from September 1. Customers can check the new rates on the official website.",
        "post": "SBI has updated fixed deposit interest rates, effective Sept 1. Check out the new rates on their website! #SBIRates #FixedDeposits"
    }
]

eval_dataset = pd.DataFrame([
    {
        "context": f"announcement: {item['prompt']}",
        "response": item["post"],
    } for item in posts
])

eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=["groundedness"],
    experiment="social-media-post-7"
)

In [58]:
prompt_template = ("Prompt: {context}. Post: {response}")
result = eval_task.evaluate(
      prompt_template=prompt_template,
      experiment_run_name="social-media-post-7"
)
evaluation_results = []
evaluation_results.append(result)

notebook_utils.display_eval_result(eval_result=result)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Prompt: {context}. Post: {response}'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 3 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 3/3 [00:00<00:00,  3.63it/s]
INFO:vertexai.evaluation._evaluation:All 3 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:0.8348894030023075 seconds


### Summary Metrics

,row_count,groundedness/mean,groundedness/std
0,3.0,1.0,0.0


### Row-based Metrics

,context,response,prompt,groundedness/explanation,groundedness/score
0,announcement: The annual cultural festival 'Ut...,Kolkata's 'Utsav' cultural festival is back! E...,Prompt: announcement: The annual cultural fest...,"The response is fully grounded, as all aspects...",1.0
1,announcement: A major power outage is expected...,Heads up Mumbai! Power outage in Bandra & Juhu...,Prompt: announcement: A major power outage is ...,The response is fully grounded as it only cont...,1.0
2,announcement: The State Bank of India has revi...,"SBI has updated fixed deposit interest rates, ...",Prompt: announcement: The State Bank of India ...,The response is fully grounded as it is direct...,1.0
